In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

Copy your score files (files ending in .sc) into the direcotry with this notebook. 
If in binder, upload into binder.

In [ ]:
listdata = []

for i in os.listdir('.'): # Reads in all score files (*.sc) within current working directory (CWD).
    if i.endswith('.sc'):
        try:
            d = pd.read_csv(i, sep='\s+')
            listdata.append(d)
        except FileNotFoundError: # This skips missing score files.
            continue

df = pd.concat(listdata) # Creates a dataframe (DF) from score file data
cst_list = []

for i in df: # Searches DF for specific constraint energy terms, this is important for docking!
    if i.endswith('_all_cst'):
        cst_list.append(df[i])
        
for j in cst_list: # This creates a new constraint column for analysis.
    df['Constraint'] = sum(cst_list)

for i in df: # Searches DF for specific interface energy term, this is important for docking!
    if i.endswith('_E_1_2'): # Interface energy alwasy ends with "_E_1_2" for single docking
        interfaceE = i
    elif i.endswith('_E_1_3'): # Interface energy alwasy ends with "_E_1_3" for multidocking
        interfaceE = i
    else: # Skips all other terms
        pass

After determining docking variables, a dataframe is created that only shows the interaction energy, score, constraint value, and pose name. The number of poses present are printed too.

In [ ]:
df_copy = df[['total_score',interfaceE,'Constraint','description']] 
df_copy = df_copy[df_copy.Constraint < 1.00] # Constraint filter, comment out to remove filter.
print(str(len(df_copy)) + " poses in df_copy")
df_copy

Filters results by interaction energy. Poses that are in the lowest 25% by interface energy are filtered.

In [ ]:
df_copy2 = df_copy.sort_values(interfaceE, ascending=True)[0:round(0.25*len(df_copy))]
print(str(len(df_10_13_11_copy2)) + " poses in df_copy2")
df_copy2

Lastly, the poses are filtered again by score. Poses that are in the lowest 25% by score are filtered.

In [ ]:
df_copy3 = df_copy2.sort_values('total_score', ascending=True)[0:round(0.25*len(df_copy2))] 
print(str(len(df_copy3)) + " posess in df_1copy3")
df_copy3.sort_values(interfaceE, ascending=True)[0:10]

This next cell will calculate a bunch of statistics with your starting and ending data.

In [ ]:
total_mean = round(df.total_score.mean(),1)
total_mean_std = round(df.total_score.std(),1)
inter_mean = round(df[interfaceE].mean(),1)
inter_mean_std = round(df[interfaceE].std(),1)
constraint_mean = round(df.Constraint.mean(),1)
constraint_mean_std = round(df.Constraint.std(),1)
filered_score_mean = round(df_copy3.total_score.mean(),1)
filered_score_mean_std = round(df_copy3.total_score.std(),1)
fitlered_inter_mean = round(df_copy3[interfaceE].mean(),1)
fitlered_inter_mean_std = round(df_copy3[interfaceE].std(),1)
fitlered_constraint_mean = round(df_copy3.Constraint.mean(),1)
fitlered_constraint_mean_std = round(df_copy3.Constraint.std(),1)
print('Starting number of poses = %s' % (str(len(df))))
print('Global Ave = %s +/- %s' % (str(total_mean), str(total_mean_std))
print('Global IntE = %s +/- %s' % (str(inter_mean), str(inter_mean_std)))
print('Global Constraint = %s +/- %s' % (str(constraint_mean), str(constraint_mean_std)))
print('Total Filtered Poses = %s ' % (str(len(df_copy3))))
print('Filtered Ave = %s +/- %s' % (str(filtered_score_mean),str(filered_score_mean_std)))
print('Filtered IntE = %s +/- %s' % (str(filtered_inter_mean),str(fitlered_inter_mean_std)))
print('Filtered Constraint = %s +/- %s' % (str(filtered_constraint_mean),str(fitlered_constraint_mean_std)))

Last cell creates a scatter plot for the final dataframe with all filtered data.

In [ ]:
df_copy3.plot.scatter(x='total_score',y=interfaceE) # Creates a scatter plot
plt.xlabel('Total Score', fontsize='12') # x-axis Label
plt.ylabel('Interface Energy', fontsize='12') # y-axis Label
plt.title('Interface Energy vs. Total Score', fontsize='14') # Plot Title
plt.axvline(total_mean,color='k', linestyle='dashed', linewidth=2) # Creates average for x-axis values
plt.axhline(inter_mean,color='k', linestyle='dashed', linewidth=2) # Creates average for y-axis values
#plt.text(df.total_score.max() - 3*df.total_score.std(), 
         #df[interfaceE].min(), 
         #'Total Score Average = %s\nInterface Energy Average = %s' 
         #% (total_mean, inter_mean), 
         #fontsize='10') # Text and text placement
#plt.axvline(-1779.7,color='r', linestyle='dashed', linewidth=1.5) #control line
#plt.axhline(6.87,color='r', linestyle='dashed', linewidth=1.5) #control line
#plt.savefig('NAME.png')